In [ ]:
import os
import json
import zipfile
import pandas as pd
from IPython.display import display
import shutil

# Pad naar de map met ZIP-bestanden en waar alles uitgepakt wordt
input_folder = "/home/nena-meijer/PyCharmMiscProject/VWS_dossiers_zip"
output_folder = "/home/nena-meijer/PyCharmMiscProject/unpacked_zipfiles"

# Maximale bestandsnaam lengte (inclusief extensie)
MAX_FILENAME_LENGTH = 100

# Kies hier het ZIP-bestand dat je wilt testen (pas de naam aan)
test_zip_filename = "Beslissing op bezwaar Wob-besluit coronamaatregelen.zip"  # Zet hier de naam van het ZIP-bestand dat je wilt testen
zip_path = os.path.join(input_folder, test_zip_filename)

# Dossier-ID starten bij 1
dossier_id = 1
extract_path = os.path.join(output_folder, f"dossier_{dossier_id}")

# Zorg dat de map bestaat
os.makedirs(extract_path, exist_ok=True)

# Data opslag
data = []

# ZIP-bestand openen en verwerken
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    for zip_info in zip_ref.infolist():
        if zip_info.is_dir():
            continue  # Skip directories

        # Bepaal originele bestandsnaam en extensie
        original_name = os.path.basename(zip_info.filename)
        file_ext = os.path.splitext(original_name)[1].lower()

        # Beperk de naam tot MAX_FILENAME_LENGTH, behoud de extensie
        base_name = os.path.splitext(original_name)[0]
        if len(base_name) > MAX_FILENAME_LENGTH:
            base_name = base_name[:MAX_FILENAME_LENGTH]
        safe_filename = f"{base_name}{file_ext}"

        # Opslaan op veilige locatie
        safe_path = os.path.join(extract_path, safe_filename)

        # Bestand veilig extraheren
        with zip_ref.open(zip_info) as source, open(safe_path, "wb") as target:
            shutil.copyfileobj(source, target)

# Metadata.json uitlezen (als die bestaat)
metadata_path = os.path.join(extract_path, "metadata.json")
published_date, decision_date = None, None
dossier_name = None
if os.path.exists(metadata_path):
    with open(metadata_path, "r", encoding="utf-8") as f:
        metadata = json.load(f)
        dossier_name = metadata.get("title")
        published_date = metadata.get("published_date")
        decision_date = metadata.get("decision_date")

# Bestandsteller starten
file_counter = 1

# Loop door alle uitgepakte bestanden
for file in os.listdir(extract_path):
    if file == "metadata.json":  # Metadata.json zelf negeren
        continue

    file_path = os.path.join(extract_path, file)
    file_ext = os.path.splitext(file)[1].lower()

    # Genereer file_id (bijv. 1-1, 1-2, etc.)
    file_id = f"{dossier_id}-{file_counter}"
    file_counter += 1  # Verhoog teller

    # Opslaan in lijst
    data.append({
        "dossier_id": dossier_id,
        "dossier_name": dossier_name,
        "file_id": file_id,
        "file_name": file,
        "file_type": file_ext,
        "dossier_published_date": published_date,
        "dossier_decision_date": decision_date
    })

# DataFrame maken en weergeven
df = pd.DataFrame(data)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)
display(df)


In [ ]:
import os
import json
import zipfile
import pandas as pd
from IPython.display import display
import shutil

# Pad naar de map met ZIP-bestanden en waar alles uitgepakt wordt
input_folder = "/home/nena-meijer/PyCharmMiscProject/VWS_dossiers_zip"
output_folder = "/home/nena-meijer/PyCharmMiscProject/unpacked_zipfiles"

# Maximale bestandsnaam lengte (inclusief extensie)
MAX_FILENAME_LENGTH = 100

# Data opslag
data = []

# Zoek alle ZIP-bestanden in de input folder
zip_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".zip")])

# Loop door alle ZIP-bestanden en verwerk ze
for dossier_id, zip_filename in enumerate(zip_files, start=1):
    zip_path = os.path.join(input_folder, zip_filename)
    extract_path = os.path.join(output_folder, f"dossier_{dossier_id}")

    # Zorg dat de map bestaat
    os.makedirs(extract_path, exist_ok=True)

    # ZIP-bestand openen en verwerken
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for zip_info in zip_ref.infolist():
            if zip_info.is_dir():
                continue  # Skip directories

            # Bepaal originele bestandsnaam en extensie
            original_name = os.path.basename(zip_info.filename)
            file_ext = os.path.splitext(original_name)[1].lower()

            # Beperk de naam tot MAX_FILENAME_LENGTH, behoud de extensie
            base_name = os.path.splitext(original_name)[0]
            if len(base_name) > MAX_FILENAME_LENGTH:
                base_name = base_name[:MAX_FILENAME_LENGTH]
            safe_filename = f"{base_name}{file_ext}"

            # Opslaan op veilige locatie
            safe_path = os.path.join(extract_path, safe_filename)

            # Bestand veilig extraheren
            with zip_ref.open(zip_info) as source, open(safe_path, "wb") as target:
                shutil.copyfileobj(source, target)

    # Metadata.json uitlezen (als die bestaat)
    metadata_path = os.path.join(extract_path, "metadata.json")
    published_date, decision_date, dossier_name = None, None, None
    if os.path.exists(metadata_path):
        with open(metadata_path, "r", encoding="utf-8") as f:
            metadata = json.load(f)
            dossier_name = metadata.get("title")
            published_date = metadata.get("published_date")
            decision_date = metadata.get("decision_date")

    # Datumconversie naar datetime-formaat (indien beschikbaar)
    published_date = pd.to_datetime(published_date, errors='coerce')
    decision_date = pd.to_datetime(decision_date, errors='coerce')

    # Bestandsteller starten
    file_counter = 1

    # Loop door alle uitgepakte bestanden
    for file in os.listdir(extract_path):
        if file == "metadata.json":  # Metadata.json zelf negeren
            continue

        file_path = os.path.join(extract_path, file)
        file_ext = os.path.splitext(file)[1].lower()

        # Genereer file_id (bijv. 1-1, 1-2, etc.)
        file_id = f"{dossier_id}-{file_counter}"
        file_counter += 1  # Verhoog teller

        # Opslaan in lijst
        data.append({
            "dossier_id": dossier_id,
            "dossier_name": dossier_name,
            "file_id": file_id,
            "file_name": file,
            "file_source": file_path,
            "file_type": file_ext,
            "dossier_published_date": published_date,
            "dossier_decision_date": decision_date
        })

# DataFrame maken en weergeven
df = pd.DataFrame(data)

# Instellingen voor brede tabellen en volledige tekstweergave
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)

# DataFrame weergeven
display(df)


In [18]:
df.to_csv("df.csv", index=False)

In [19]:
import pandas as pd
from IPython.display import display

# CSV inlezen
df = pd.read_csv("df.csv")

# Filter alleen relevante kolommen
df_filtered = df[["dossier_id", "dossier_name", "file_name", "dossier_published_date", "dossier_decision_date"]]

# Verwijder metadata.json en groepeer per dossier_id
df_aggregated = (
    df_filtered[df_filtered["file_name"] != "metadata.json"]
    .groupby(["dossier_id", "dossier_name", "dossier_published_date", "dossier_decision_date"])
    .agg(document_count=("file_name", "count"))  # Tel documenten per dossier
    .reset_index()
)

# Instellingen om lange tekst volledig te tonen
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)

# DataFrame weergeven in Jupyter Notebook
display(df_aggregated)

# Opslaan als CSV
df_aggregated.to_csv("df_dossiers.csv", index=False)

print("✅ Geaggregeerde CSV opgeslagen als 'df_dossiers.csv'")


,dossier_id,dossier_name,dossier_published_date,dossier_decision_date,document_count
0,1,(4e deel) Besluit Woo-verzoek Covid-19 aanpak DOC-19 Q4 2020,2024-02-05,2023-07-29,229
1,2,Woo-besluit aangaande kaders werkproces Wob-verzoeken,2024-03-22,2023-03-20,83
2,3,1e deel Besluit Woo verzoek Covid-19 aanpak ACC Q4 2020,2024-02-07,2022-08-18,208
3,4,2e deel Besluit Woo verzoek Covid-19 aanpak ACC Q1 2021,2024-02-08,2023-02-17,239
4,5,3e deel Besluit Woo verzoek Covid-19 aanpak ACC-19 April-Mei 2021,2024-02-07,2022-05-12,144
...,...,...,...,...,...
215,216,Woo-deelbesluit aangaande overeenkomsten PCR-laboratoria,2024-06-06,2024-03-27,68
216,217,Woo-deelbesluit aangaande vaccinatie aansprakelijkheid Deel II,2024-10-09,2024-10-02,116
217,218,Woo-deelbesluit aangaande ‘Capaciteit van ziekenhuizen IC-capaciteit',2024-10-19,2024-09-12,199
218,219,Woo-deelbesluit aangaande ‘de inkoop van persoonlijke beschermingsmiddelen tijdens de coronacrisis en het onderzoek daarnaar’,2024-04-05,2024-02-14,2695


✅ Geaggregeerde CSV opgeslagen als 'df_dossiers.csv'


In [25]:
import pandas as pd
from IPython.display import display

# CSV inlezen
df = pd.read_csv("df.csv")

# Verwijder spaties in kolomnamen (indien nodig)
df.columns = df.columns.str.strip()

# Controleer of de verwachte kolommen aanwezig zijn
expected_columns = ["dossier_id", "file_name", "file_type"]
missing_columns = [col for col in expected_columns if col not in df.columns]

if missing_columns:
    print(f"❌ De volgende kolommen ontbreken in de CSV: {missing_columns}")
else:
    # Selecteer relevante kolommen en hernoem file_name naar document_title
    df_documents = df[["dossier_id", "file_name", "file_type"]].rename(
        columns={"file_name": "document_title", "file_type": "document_type"}
    )

    # Verwijder metadata.json
    df_documents = df_documents[df_documents["document_title"] != "metadata.json"]

    # Verwijder de punt uit document_type (extensie)
    df_documents["document_type"] = df_documents["document_type"].str.replace(".", "", regex=False)

    # Document-ID toevoegen (per dossier_id een oplopende teller)
    df_documents["document_id"] = (
        df_documents.groupby("dossier_id").cumcount() + 1  # Start teller bij 1 per dossier
    ).astype(str)

    # Combineer document_id met dossier_id
    df_documents["document_id"] = df_documents["dossier_id"].astype(str) + "-" + df_documents["document_id"]

    # Kolommen opnieuw ordenen
    df_documents = df_documents[["dossier_id", "document_id", "document_title", "document_type"]]

    # Opslaan als nieuwe CSV
    df_documents.to_csv("df_documents.csv", index=False)

    # DataFrame weergeven
    display(df_documents)
    print("✅ Geëxporteerd als 'df_documents.csv'")


,dossier_id,document_id,document_title,document_type
0,1,1-1,VWS-WC-098-DOC201208J-Concept_Platte_tekst_herijkte_routekaart.pdf,pdf
1,1,1-2,VWS-WC-098-201130D-Actiepuntenlijst_DOC-19_-_27_november_2020.pdf,pdf
2,1,1-3,VWS-WC-098-201127F-Presentatie_verdiepingssessie_vaccinatiestrategie.pdf,pdf
3,1,1-4,VWS-WC-098-201215F-Implementatie_gedragsinterventies_DGSC-19_en_RRT.pdf,pdf
4,1,1-5,VWS-WC-098-201006A-Agenda_DOC-19_6_oktober_2020.pdf,pdf
...,...,...,...,...
109029,220,220-5,VWS-WC-107-2a-2020_RV_0022_Masterplan_emissieloze_maritieme_sector.pdf,pdf
109030,220,220-6,VWS-WC-107-1-FW_Masterplan_voor_een_emissieloze_maritieme_sector.pdf,pdf
109031,220,220-7,VWS-WC-107-4-video-overleg_Corona_herstelfonds.pdf,pdf
109032,220,220-8,VWS-WC-107-3-Vraag_om_informatie_over_RFF_in_relatie_tot_emissiearme_kotter_visserij.pdf,pdf


✅ Geëxporteerd als 'df_documents.csv'


In [2]:
import time

def scrape_document_metadata(dossier_url):
    """Scrapet documentmetadata van alle pagina's binnen een dossier met een crawl delay."""

    # Extract dossier ID uit de URL
    dossier_id_match = re.search(r'/dossier/([^/]+)', dossier_url)
    dossier_id_str = dossier_id_match.group(1) if dossier_id_match else None

    metadata_list = []
    seen_documents = set()
    page = 1
    max_pages = 50  # Failsafe om te voorkomen dat hij oneindig blijft lopen

    while page <= max_pages:
        paged_url = f"{dossier_url}?prefix=VWS-WC&dossierId={dossier_id_str}&pu={page}#tabcontrol-1"
        response = requests.get(paged_url)
        if response.status_code != 200:
            print(f"Fout bij ophalen van {paged_url}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        table_rows = soup.select(".woo-table tbody tr")
        if not table_rows:
            print(f"Geen documenten gevonden op pagina {page}, stoppen met scrapen.")
            break

        new_documents_found = False

        for row in table_rows:
            try:
                # Document ID ophalen uit de eerste kolom
                doc_id_tag = row.select_one("td:nth-of-type(1)")
                doc_id = doc_id_tag.text.strip() if doc_id_tag else None

                # Document URL ophalen
                doc_link_tag = row.select_one("td:nth-of-type(3) a")
                doc_url = doc_link_tag["href"] if doc_link_tag else ""

                if not doc_id or doc_id in seen_documents:
                    continue

                seen_documents.add(doc_id)
                new_documents_found = True

                # Documenttype
                doc_type_tag = row.select_one("td:nth-of-type(2) use")
                doc_type_key = doc_type_tag["xlink:href"].split("#")[-1] if doc_type_tag else "unknown"
                doc_type = DOCUMENT_TYPE_MAP.get(doc_type_key, "Onbekend")

                # Documentdatum correct ophalen
                doc_date_tag = row.select_one("td:nth-of-type(4) time")
                doc_date = doc_date_tag["datetime"] if doc_date_tag else None

                metadata_list.append({
                    "document_id": doc_id,
                    "document_sourcetype": doc_type,
                    "document_date": doc_date,
                    "dossier_id": dossier_id
                })
            except Exception as e:
                print(f"Fout bij parsen: {e}")

        if not new_documents_found:
            print(f"Geen nieuwe documenten meer gevonden op pagina {page}, stoppen met scrapen.")
            break

        page += 1
        time.sleep(1)  # Crawl delay na elke pagina scrape

    for doc in metadata_list:
        print(doc)

    return metadata_list


In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Mapping van documenttypes
DOCUMENT_TYPE_MAP = {
    "unknown": "Onbekend",
    "pdf": "PDF",
    "email": "E-mailbericht",
    "presentation": "Presentatie",
    "doc": "Word-document",
    "chat": "Chatbericht",
    "image": "Afbeelding",
    "spreadsheet": "Spreadsheet"
}

# Inlezen van de bestaande documentgegevens
df_documents = pd.read_csv("df_documents_dates.csv", dtype={"document_id": str})
df_dossiers = pd.read_csv("df_dossiers_final.csv", dtype={"dossier_id": int})

# Handmatig dossier_id invoeren
dossier_id = 27
dossier = df_dossiers[df_dossiers["dossier_id"] == dossier_id].iloc[0]
dossier_url = dossier["dossier_sourceURL"]

print(f"Scrapen: {dossier_url}")
document_metadata = scrape_document_metadata(dossier_url)

# Omzetten naar dataframe
df_metadata = pd.DataFrame(document_metadata)

def extract_prefix(document_title):
    """Probeert het juiste dossierprefix te extraheren uit de document_title."""

    # Specifieke uitzondering: als het document begint met "VWS-WC-19.00-1-", dan is de prefix gewoon "VWS-WC"
    # if document_title.startswith("VWS-WC-19.00-1-"):
    #     return "VWS-WC-19.00-1-"

    # Standaardpatronen voor prefixes
    match = re.match(r"^(VWS-WC-\d{3}|VWS-WOO-[A-Za-z0-9]+|VWS-WC)", document_title)
    return match.group(1) if match else None

def match_documents(df_documents, df_metadata, dossier_id):
    """Matcht gescrapete metadata met documenten op basis van dossierprefix en document_id."""
    df_documents_dossier = df_documents[df_documents["dossier_id"] == dossier_id]

    if df_documents_dossier.empty:
        print(f"⚠️ Geen documenten gevonden voor dossier_id {dossier_id}")
        return df_documents

    # Zoek het eerste document dat GEEN generieke naam heeft
    sample_document_title = None
    for title in df_documents_dossier["document_title"]:
        if not title.lower().startswith("besluitbrief"):  # Voeg andere generieke namen toe indien nodig
            sample_document_title = title
            break

    if not sample_document_title:
        print(f"❌ Geen geldig document gevonden om een prefix uit te halen voor dossier {dossier_id}.")
        return df_documents

    dossier_prefix = extract_prefix(sample_document_title)

    if not dossier_prefix:
        print(f"❌ Geen geldig dossierprefix gevonden in: {sample_document_title}")
        return df_documents

    print(f"🔎 Dossierprefix bepaald als: {dossier_prefix}")

    for i, doc_row in df_documents_dossier.iterrows():
        document_title = doc_row["document_title"]
        print(f"\n---\n🔍 Matching document: {document_title}")

        for _, meta_row in df_metadata.iterrows():
            meta_doc_id = str(meta_row["document_id"]).strip()  # Zorgt dat het ID een string is
            expected_identifier = f"{dossier_prefix}-{meta_doc_id}"  # Verwachte match

            # Exacte match met regex
            if re.search(rf"\b{re.escape(expected_identifier)}\b", document_title):
                print(f"  ✅ MATCH GEVONDEN! Updaten document {meta_doc_id}")

                df_documents.at[i, "document_sourcetype"] = meta_row["document_sourcetype"]
                df_documents.at[i, "document_date"] = meta_row["document_date"]

                # Print de waarde die in de CSV wordt gezet
                print(f"  📌 In CSV zetten → Type: {meta_row['document_sourcetype']}, Datum: {meta_row['document_date']}")
                break  # Stop met zoeken zodra de match is gevonden

    return df_documents

df_documents = match_documents(df_documents, df_metadata, dossier_id)

# Opslaan van de verrijkte dataset
df_documents.to_csv("df_documents_dates.csv", index=False)
print(f"Scraping en verrijking voltooid voor dossier {dossier_id}!")
print(dossier_url)


Scrapen: https://open.minvws.nl/dossier/VWS-WC/036
Geen nieuwe documenten meer gevonden op pagina 2, stoppen met scrapen.
{'document_id': 'Openbaarmaking', 'document_sourcetype': 'Onbekend', 'document_date': None, 'dossier_id': 27}
{'document_id': 'Wob-besluit', 'document_sourcetype': 'Onbekend', 'document_date': None, 'dossier_id': 27}
{'document_id': 'besluit (3).pdf (PDF, 2.22 MB)', 'document_sourcetype': 'Onbekend', 'document_date': None, 'dossier_id': 27}
{'document_id': 'ministerie van Financiën', 'document_sourcetype': 'Onbekend', 'document_date': None, 'dossier_id': 27}
{'document_id': 'Februari - juni 2020', 'document_sourcetype': 'Onbekend', 'document_date': None, 'dossier_id': 27}
{'document_id': '9 februari 2021', 'document_sourcetype': 'Onbekend', 'document_date': None, 'dossier_id': 27}
{'document_id': "117 documenten,                        \n                                                    182 pagina's\n                            \n                                  